<a href="https://colab.research.google.com/github/mavillan/jane-street-market-prediction/blob/main/tuning/cnn1d-optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -Iv scikit-learn==0.23.2 > /dev/null 2>&1
!pip install optuna > /dev/null 2>&1

In [3]:
import copy
import os
import numpy as np 
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader

# custom modules
import sys
sys.path.append("/content/drive/MyDrive/kaggle/janestreet")
from torch_utils import Monitor, train_step, valid_step

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

***
### utility functions

In [4]:
def utility_score(date, weight, resp, action):
    """
    Fast computation of utility score
    """
    date = date.astype(int)
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return -u

In [5]:
def cat_encoder(X):
    """
    Fast one-hot encoding of feature_0
    """
    X["feature_00"] = 0
    idx00 = X.query("feature_0 == -1").index
    X.loc[idx00,"feature_00"] = 1
    
    X["feature_01"] = 0
    idx01 = X.query("feature_0 == 1").index
    X.loc[idx01,"feature_01"] = 1
    
    return X.iloc[:,1:]

In [6]:
def show_metrics(monitor):
    x = np.arange(len(monitor.train_loss))
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(21, 7))
    
    ax1 = axes[0]
    ax2 = ax1.twinx()
    ax1.plot(x, monitor.train_loss, 'go-', label="train_loss")
    ax2.plot(x, monitor.train_metric, 'ro-', label="train_metric")
    plt.legend(loc="best")
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('loss')
    ax1.set_title("Training")
    plt.grid()
    
    ax1 = axes[1]
    ax2 = ax1.twinx()
    ax1.plot(x, monitor.valid_loss, 'go-', label="valid_loss")
    ax2.plot(x, monitor.valid_metric, 'ro-', label="valid_metric")
    plt.legend(loc="best")
    ax1.set_xlabel('epochs')
    ax2.set_ylabel('metric')
    ax1.set_title("Validation")
    plt.grid()
    
    plt.show()

***
### preparing the data

In [7]:
root = Path("/content/drive/MyDrive/kaggle/janestreet/preprocessing/")

train = pd.read_parquet(root/"train.parquet")
features = pd.read_parquet(root/"features.parquet")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 143 entries, date to w4
dtypes: float32(140), int16(1), int32(1), int8(1)
memory usage: 1.3 GB


In [8]:
train = train.query("date > 85").query("weight > 0").reset_index(drop=True)

input_features = [col for col in train.columns if "feature" in col]
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
w_cols = ["w", "w1", "w2", "w3", "w4"]

X_dset = train.loc[:,input_features].copy()
y_dset = (train.loc[:,resp_cols] > 0).astype(int).copy()
w_dset = train.loc[:, w_cols].copy()
dwr_dset = train.loc[:, ["date","weight","resp"]].copy()

In [9]:
%%time 

with open("/content/drive/MyDrive/kaggle/janestreet/imputer/imputer_f0m1.pickle", "rb") as file:
    imputer_f0m1 = pickle.load(file)
    file.close()
    
with open("/content/drive/MyDrive/kaggle/janestreet/imputer/imputer_f0p1.pickle", "rb") as file:
    imputer_f0p1 = pickle.load(file)
    file.close()

idx_f0m1 = X_dset.query("feature_0 == -1").index
X_dset.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_dset.loc[idx_f0m1, input_features[1:]])

idx_f0p1 = X_dset.query("feature_0 ==  1").index
X_dset.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_dset.loc[idx_f0p1, input_features[1:]])

CPU times: user 38.7 s, sys: 6.12 s, total: 44.8 s
Wall time: 41.2 s


In [10]:
X_dset = cat_encoder(X_dset)
input_features = X_dset.columns.tolist()

***
### model definition

In [11]:
class CNN1D(nn.Module):
    def __init__(self, input_dim, output_dim, sign_size=16, cha_1=32, cha_2=64, cha_3=64, 
                 dropout_in1=0.2, dropout_in2=0.2, dropout_mid=0.3, dropout_out=0.2):
        super(CNN1D, self).__init__()

        hidden_size = sign_size*cha_1
        sign_size1 = sign_size
        sign_size2 = sign_size//2
        output_size1 = (sign_size//4) * cha_3
        output_size2 = output_size1//8

        self.hidden_size = hidden_size
        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.sign_size1 = sign_size1
        self.sign_size2 = sign_size2
        self.output_size1 = output_size1
        self.output_size2 = output_size2
        self.dropout_in1 = dropout_in1
        self.dropout_in2 = dropout_in2
        self.dropout_mid = dropout_mid
        self.dropout_out = dropout_out

        self.batch_norm1 = nn.BatchNorm1d(input_dim)
        self.dropout1 = nn.Dropout(dropout_in1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(input_dim, hidden_size, bias=False))

        # 1st conv layer
        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(dropout_in2)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = sign_size2)

        # 2nd conv layer
        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(dropout_in2)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        # 3rd conv layer
        self.batch_norm_c3 = nn.BatchNorm1d(cha_2)
        self.dropout_c3 = nn.Dropout(dropout_mid)
        self.conv3 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        # 4th conv layer
        self.batch_norm_c4 = nn.BatchNorm1d(cha_2)
        self.dropout_c4 = nn.Dropout(dropout_mid/2)
        self.conv4 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

        self.max_po_c4 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm2 = nn.BatchNorm1d(output_size1)
        self.dropout2 = nn.Dropout(dropout_out)
        self.dense2 = nn.utils.weight_norm(nn.Linear(output_size1, output_size2, bias=False))

        self.batch_norm3 = nn.BatchNorm1d(output_size2)
        self.dropout3 = nn.Dropout(dropout_out/2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(output_size2, output_dim, bias=False))

    def forward(self, x):

        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))

        x = x.reshape(x.shape[0], self.cha_1, self.sign_size1)

        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c3(x)
        x = self.dropout_c3(x)
        x = F.relu(self.conv3(x))

        x = self.batch_norm_c4(x)
        x = self.dropout_c4(x)
        x = F.relu(self.conv4(x))
        x =  x * x_s

        x = self.max_po_c4(x)

        x = self.flt(x)

        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x

In [12]:
class BCELabelSmoothing(nn.Module):
    def __init__(self, label_smoothing=0.0):
        super(BCELabelSmoothing, self).__init__()
        self.label_smoothing = label_smoothing
        self.bce_loss = torch.nn.functional.binary_cross_entropy_with_logits
        
    def forward(self, prediction, target, weight=None):
        target_smooth = target*(1.0 - self.label_smoothing) + 0.5*self.label_smoothing
        if weight is None:
            loss = self.bce_loss(prediction, target_smooth, reduction="mean")
        else:
            loss = self.bce_loss(prediction, target_smooth, weight, reduction="sum") / torch.sum(weight)
        return loss

bce_loss = BCELabelSmoothing(label_smoothing=1e-2)

***
### Hyperparameter tuning

In [13]:
# 80% rows for train & 20% for valid
train_idx = train.query("date < 430").index
valid_idx = train.query("date >= 430").index

train_dset = TensorDataset(torch.tensor(X_dset.loc[train_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[train_idx].values, dtype=torch.float),
                          )

valid_dset = TensorDataset(torch.tensor(X_dset.loc[valid_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[valid_idx].values, dtype=torch.float),
                          )

dataset_sizes = {'train': len(train_dset), 'valid': len(valid_dset)}
train_dataloader = DataLoader(train_dset, batch_size=2048, shuffle=True, num_workers=2)
valid_dataloader = DataLoader(valid_dset, batch_size=len(valid_dset), shuffle=False, num_workers=2)

print("Number of step per epoch:", len(train_dset)//2048)

Number of step per epoch: 612


In [14]:
if os.path.exists(f"/content/drive/MyDrive/kaggle/janestreet/logs/cnn1d_tuning.csv"):
    logger = open(f"/content/drive/MyDrive/kaggle/janestreet/logs/cnn1d_tuning.csv", "a")
else:
    logger = open(f"/content/drive/MyDrive/kaggle/janestreet/logs/cnn1d_tuning.csv", "w")
    logger.write("trial;params;loss;metric;loss_hist;metric_hist\n")

In [21]:
default_params = dict(
    sign_size=22,
    cha_1=16,
    cha_2=32,
    cha_3=32,
)

def objective(trial):
    dropout_in1 = trial.suggest_discrete_uniform("dropout_in1", 0.1, 0.4, 0.05)
    dropout_in2 = trial.suggest_discrete_uniform("dropout_in2", 0.1, 0.4, 0.05) 
    dropout_mid = trial.suggest_discrete_uniform("dropout_mid", 0.1, 0.4, 0.05)
    dropout_out = trial.suggest_discrete_uniform("dropout_out", 0.1, 0.4, 0.05)
    #sign_size = int(trial.suggest_discrete_uniform("sign_size", 8, 24, 2))
    #channels = int(trial.suggest_discrete_uniform("channels", 16, 64, 8))
    nn_kwargs = {
        **default_params,
        "input_dim":len(input_features),
        "output_dim":len(resp_cols),
        "dropout_in1":dropout_in1,
        "dropout_in2":dropout_in2,
        "dropout_mid":dropout_mid,
        "dropout_out":dropout_out,
    }

    # other hyperparams
    weight_decay = 10 ** trial.suggest_int("weight_decay", -5, -3)
    max_lr = trial.suggest_discrete_uniform("max_lr", 0.01, 0.10, 0.02)
    #pct_start = trial.suggest_discrete_uniform("pct_start", 0.1, 0.5, 0.1)
    pct_start = 0.1

    sampled_params = {
        **nn_kwargs,
        "weight_decay":weight_decay,
        "pct_start":pct_start,
        "max_lr":max_lr,
    }
    print("-"*80)
    print("sampled_params:", sampled_params)

    model = CNN1D(**nn_kwargs)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=0.9, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=max_lr,
        epochs=50,
        pct_start=pct_start, 
        anneal_strategy='cos', 
        cycle_momentum=True, 
        base_momentum=0.8, 
        max_momentum=0.9, 
        div_factor=1e1,
        final_div_factor=1e0,
        steps_per_epoch=len(train_dataloader),
        verbose=False)
    
    monitor = Monitor(
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        patience=10,
        metric_fn=utility_score,
        experiment_name=f'CNN1D',
        num_epochs=50,
        dataset_sizes=dataset_sizes,
        early_stop_on_metric=False,
        lower_is_better=True)
    
    for epoch in monitor.iter_epochs:
        train_step(model, train_dataloader, optimizer, monitor, bce_loss, scheduler=scheduler, clip_value=None)    
        early_stop = valid_step(model, valid_dataloader, optimizer, monitor, bce_loss)
        if early_stop: break

    logger.write(f"{trial.number};{sampled_params};{monitor.best_loss};{monitor.best_metric};{monitor.valid_loss[-10:]};{monitor.valid_metric[-10:]}\n")
    logger.flush()

    print(f"best_valid_loss: {monitor.best_loss} - best_valid_metric: {monitor.best_metric}")
    return monitor.best_loss

In [22]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, timeout=43200, show_progress_bar=False)

[I 2021-02-17 05:28:45,823] A new study created in memory with name: no-name-eca8f5c7-0c8e-492b-bd74-ed7592517e8b
/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:548: UserWarning:

The distribution is specified by [0.01, 0.1] and q=0.02, but the range is not divisible by `q`. It will be replaced by [0.01, 0.09].



--------------------------------------------------------------------------------
sampled_params: {'sign_size': 22, 'cha_1': 16, 'cha_2': 32, 'cha_3': 32, 'input_dim': 131, 'output_dim': 5, 'dropout_in1': 0.30000000000000004, 'dropout_in2': 0.25, 'dropout_mid': 0.15000000000000002, 'dropout_out': 0.15000000000000002, 'weight_decay': 1e-05, 'pct_start': 0.1, 'max_lr': 0.06999999999999999}


[I 2021-02-17 05:48:18,143] Trial 0 finished with value: 0.6859996914863586 and parameters: {'dropout_in1': 0.30000000000000004, 'dropout_in2': 0.25, 'dropout_mid': 0.15000000000000002, 'dropout_out': 0.15000000000000002, 'weight_decay': -5, 'max_lr': 0.06999999999999999}. Best is trial 0 with value: 0.6859996914863586.



best_valid_loss: 0.6859996914863586 - best_valid_metric: -2396.2469016499385
--------------------------------------------------------------------------------
sampled_params: {'sign_size': 22, 'cha_1': 16, 'cha_2': 32, 'cha_3': 32, 'input_dim': 131, 'output_dim': 5, 'dropout_in1': 0.30000000000000004, 'dropout_in2': 0.1, 'dropout_mid': 0.4, 'dropout_out': 0.15000000000000002, 'weight_decay': 0.0001, 'pct_start': 0.1, 'max_lr': 0.01}


[I 2021-02-17 06:20:53,029] Trial 1 finished with value: 0.6877446174621582 and parameters: {'dropout_in1': 0.30000000000000004, 'dropout_in2': 0.1, 'dropout_mid': 0.4, 'dropout_out': 0.15000000000000002, 'weight_decay': -4, 'max_lr': 0.01}. Best is trial 0 with value: 0.6859996914863586.


...


best_valid_loss: 0.6863821744918823 - best_valid_metric: -2416.430895697491
--------------------------------------------------------------------------------
sampled_params: {'sign_size': 22, 'cha_1': 16, 'cha_2': 32, 'cha_3': 32, 'input_dim': 131, 'output_dim': 5, 'dropout_in1': 0.15000000000000002, 'dropout_in2': 0.25, 'dropout_mid': 0.35, 'dropout_out': 0.25, 'weight_decay': 1e-05, 'pct_start': 0.1, 'max_lr': 0.06999999999999999}


[I 2021-02-17 17:14:02,637] Trial 32 finished with value: 0.6860525012016296 and parameters: {'dropout_in1': 0.15000000000000002, 'dropout_in2': 0.25, 'dropout_mid': 0.35, 'dropout_out': 0.25, 'weight_decay': -5, 'max_lr': 0.06999999999999999}. Best is trial 11 with value: 0.6850374937057495.



best_valid_loss: 0.6860525012016296 - best_valid_metric: -2525.904877007705
--------------------------------------------------------------------------------
sampled_params: {'sign_size': 22, 'cha_1': 16, 'cha_2': 32, 'cha_3': 32, 'input_dim': 131, 'output_dim': 5, 'dropout_in1': 0.2, 'dropout_in2': 0.35, 'dropout_mid': 0.4, 'dropout_out': 0.2, 'weight_decay': 0.0001, 'pct_start': 0.1, 'max_lr': 0.05}


[I 2021-02-17 17:43:16,988] Trial 33 finished with value: 0.685244083404541 and parameters: {'dropout_in1': 0.2, 'dropout_in2': 0.35, 'dropout_mid': 0.4, 'dropout_out': 0.2, 'weight_decay': -4, 'max_lr': 0.05}. Best is trial 11 with value: 0.6850374937057495.



best_valid_loss: 0.685244083404541 - best_valid_metric: -2509.8785526952315


***